In [1]:
# --- 1. Setup and Imports ---

import sys
import subprocess
import importlib

print("Assuming packages are pre-installed from previous notebooks.")

import os
import gc
import random
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import time

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast

import segmentation_models_pytorch as smp
from sklearn.model_selection import KFold
import albumentations as A
from albumentations.pytorch import ToTensorV2

Assuming packages are pre-installed from previous notebooks.


In [2]:
# --- 2. Configuration for Fine-Tuning ---
class CFG:
    # General
    DEBUG = False
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    SEED = 42

    # Data Paths
    TRAIN_PATH = 'train'
    TRAIN_FRAGMENTS = ['1', '2']

    # Model
    BACKBONE = 'timm-efficientnet-b4'
    IN_CHANS = 25
    MODEL_LOAD_PATH = 'best_robust_model.pth'  # Load the robust model
    MODEL_SAVE_PATH = 'best_finetuned_model.pth' # Save the new fine-tuned model

    # Fine-Tuning (Expert Plan A)
    EPOCHS = 8
    BATCH_SIZE = 4  # Reduced from 8 to resolve OOM error
    LR_ENCODER = 1e-5
    LR_DECODER = 5e-5
    WD = 1e-4
    PATIENCE = 3
    VALIDATION_FRAGMENT = '2'

    # Data Reading & Tiling (Consistent with previous training)
    Z_START = 20
    Z_END = 44
    TILE_SIZE = 256
    STRIDE = TILE_SIZE // 2

    # Augmentation (Keep forcing TIF-only learning)
    SIMULATE_IR_PROB = 1.0

if CFG.DEBUG:
    CFG.EPOCHS = 2
    CFG.TRAIN_FRAGMENTS = ['1']

def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(CFG.SEED)
print(f"Fine-tuning on fragments: {[f for f in CFG.TRAIN_FRAGMENTS if f != CFG.VALIDATION_FRAGMENT]}")
print(f"Validating on fragment: {CFG.VALIDATION_FRAGMENT}")
print(f"Loading model from: {CFG.MODEL_LOAD_PATH}")
print(f"Will save best model to: {CFG.MODEL_SAVE_PATH}")

Fine-tuning on fragments: ['1']
Validating on fragment: 2
Loading model from: best_robust_model.pth
Will save best model to: best_finetuned_model.pth


In [3]:
# --- 3. Data Loading and Augmentation (Consistent with Robust Training) ---

def get_transforms(is_train=True):
    """Returns a set of augmentations."""
    if is_train:
        return A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            ToTensorV2(transpose_mask=True),
        ])
    else:
        return A.Compose([
            ToTensorV2(transpose_mask=True),
        ])

class VesuviusDataset(Dataset):
    def __init__(self, tiles, is_train=True):
        self.tiles = tiles
        self.is_train = is_train
        self.transforms = get_transforms(is_train)
        self.fragment_data_cache = {}

    def __len__(self):
        return len(self.tiles)

    def _load_fragment_data(self, fragment_id):
        """Loads and caches the components for a given fragment."""
        print(f"  (Cache miss. Loading and normalizing fragment {fragment_id}...)")
        
        tif_stack = []
        for i in range(CFG.Z_START, CFG.Z_END):
            path = os.path.join(CFG.TRAIN_PATH, fragment_id, 'surface_volume', f'{i:02}.tif')
            slice_img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
            if slice_img is None: raise FileNotFoundError(f"TIF file not found: {path}")
            
            p1, p99 = np.percentile(slice_img, [1, 99])
            slice_norm = (slice_img.astype(np.float32) - p1) / (p99 - p1 + 1e-6)
            slice_norm = np.clip(slice_norm, 0, 1)
            tif_stack.append(slice_norm)
        tif_stack = np.stack(tif_stack, axis=-1)

        ir_path = os.path.join(CFG.TRAIN_PATH, fragment_id, 'ir.png')
        ir_image = cv2.imread(ir_path, cv2.IMREAD_UNCHANGED)
        
        if ir_image is not None:
            p1, p99 = np.percentile(ir_image, [1, 99])
            ir_norm = (ir_image.astype(np.float32) - p1) / (p99 - p1 + 1e-6)
            ir_norm = np.clip(ir_norm, 0, 1)
        else:
            ir_norm = np.mean(tif_stack, axis=-1)
            
        labels_path = os.path.join(CFG.TRAIN_PATH, fragment_id, 'inklabels.png')
        labels = (cv2.imread(labels_path, cv2.IMREAD_GRAYSCALE) > 0).astype(np.uint8)
        
        self.fragment_data_cache[fragment_id] = (tif_stack, ir_norm, labels)

    def __getitem__(self, idx):
        fragment_id, (y, x) = self.tiles[idx]
        
        if fragment_id not in self.fragment_data_cache:
            self._load_fragment_data(fragment_id)
        
        tif_stack, ir_norm, labels = self.fragment_data_cache[fragment_id]
        
        tif_tile = tif_stack[y:y + CFG.TILE_SIZE, x:x + CFG.TILE_SIZE, :]
        label_tile = labels[y:y + CFG.TILE_SIZE, x:x + CFG.TILE_SIZE]

        # Per-sample IR simulation decision
        simulate_ir = (self.is_train and random.random() < CFG.SIMULATE_IR_PROB) or (not self.is_train)
        
        if simulate_ir:
            ir_tile = np.mean(tif_tile, axis=-1, keepdims=True)
        else:
            ir_tile = ir_norm[y:y + CFG.TILE_SIZE, x:x + CFG.TILE_SIZE]
            ir_tile = np.expand_dims(ir_tile, axis=-1)

        image_tile = np.concatenate([tif_tile, ir_tile], axis=-1)
        
        transformed = self.transforms(image=image_tile, mask=label_tile)
        image = transformed['image']
        mask = transformed['mask'].unsqueeze(0).float()
        
        return image, mask

In [4]:
# --- 4. Prepare Datasets (Lazy Loading) ---

def make_tiles(labels, roi_mask):
    """Generates tile coordinates with balanced sampling based on labels and ROI."""
    tiles = []
    ink_tiles = []
    no_ink_tiles = []
    
    img_height, img_width = roi_mask.shape
    
    for y in range(0, img_height - CFG.TILE_SIZE + 1, CFG.STRIDE):
        for x in range(0, img_width - CFG.TILE_SIZE + 1, CFG.STRIDE):
            # Check if the tile is within the ROI
            if (roi_mask[y:y+CFG.TILE_SIZE, x:x+CFG.TILE_SIZE] > 0).mean() > 0.5:
                label_tile = labels[y:y+CFG.TILE_SIZE, x:x+CFG.TILE_SIZE]
                if label_tile.sum() > 0:
                    ink_tiles.append((y, x))
                else:
                    no_ink_tiles.append((y, x))
    
    # Balance the dataset
    num_ink_tiles = len(ink_tiles)
    num_no_ink_to_sample = min(len(no_ink_tiles), num_ink_tiles * 2)
    
    tiles.extend(ink_tiles)
    if num_no_ink_to_sample > 0:
        tiles.extend(random.sample(no_ink_tiles, num_no_ink_to_sample))
    
    print(f"  Generated {len(tiles)} tiles ({num_ink_tiles} with ink, {num_no_ink_to_sample} without ink). Randomizing tile order.")
    random.shuffle(tiles)
    return tiles

def get_dataloaders():
    """Prepares and returns train and validation dataloaders using lazy loading."""
    
    # --- Training Data ---
    print("--- Preparing Training Tiles ---")
    train_tiles = []
    train_fragment_ids = [f for f in CFG.TRAIN_FRAGMENTS if f != CFG.VALIDATION_FRAGMENT]
    for fragment_id in train_fragment_ids:
        print(f"Processing fragment {fragment_id} for tile generation...")
        labels = (cv2.imread(os.path.join(CFG.TRAIN_PATH, fragment_id, 'inklabels.png'), cv2.IMREAD_GRAYSCALE) > 0).astype(np.uint8)
        roi_mask = cv2.imread(os.path.join(CFG.TRAIN_PATH, fragment_id, 'mask.png'), cv2.IMREAD_GRAYSCALE)
        
        fragment_tiles = make_tiles(labels, roi_mask)
        train_tiles.extend([(fragment_id, tile) for tile in fragment_tiles])

    train_dataset = VesuviusDataset(train_tiles, is_train=True)
    train_loader = DataLoader(train_dataset, batch_size=CFG.BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)

    # --- Validation Data ---
    print("\n--- Preparing Validation Tiles ---")
    print(f"Processing fragment {CFG.VALIDATION_FRAGMENT} for tile generation...")
    val_labels = (cv2.imread(os.path.join(CFG.TRAIN_PATH, CFG.VALIDATION_FRAGMENT, 'inklabels.png'), cv2.IMREAD_GRAYSCALE) > 0).astype(np.uint8)
    val_roi_mask = cv2.imread(os.path.join(CFG.TRAIN_PATH, CFG.VALIDATION_FRAGMENT, 'mask.png'), cv2.IMREAD_GRAYSCALE)
    
    val_tiles = make_tiles(val_labels, val_roi_mask)
    val_tiles_with_id = [(CFG.VALIDATION_FRAGMENT, tile) for tile in val_tiles]
    
    val_dataset = VesuviusDataset(val_tiles_with_id, is_train=False)
    val_loader = DataLoader(val_dataset, batch_size=CFG.BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
    
    return train_loader, val_loader

In [5]:
# --- 5. Model, Precision-Focused Loss, and Fine-Tuning Loop ---

# Tversky Loss for focusing on precision (high beta for F-beta)
class TverskyLoss(nn.Module):
    def __init__(self, alpha=0.5, beta=0.5, smooth=1e-6):
        super(TverskyLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.smooth = smooth

    def forward(self, y_pred, y_true):
        y_pred = torch.sigmoid(y_pred)
        
        # True Positives, False Positives & False Negatives
        TP = (y_pred * y_true).sum()
        FP = ((1 - y_true) * y_pred).sum()
        FN = (y_true * (1 - y_pred)).sum()
       
        tversky = (TP + self.smooth) / (TP + self.alpha * FP + self.beta * FN + self.smooth)  
        
        return 1 - tversky

# Combined Loss Function (BCE + Tversky)
bce_loss = smp.losses.SoftBCEWithLogitsLoss()
# For F0.5 score, beta_sq = 0.25. Tversky alpha+beta=1. So alpha=0.75, beta=0.25
tversky_loss = TverskyLoss(alpha=0.75, beta=0.25)
def criterion(y_pred, y_true):
    return 0.5 * bce_loss(y_pred, y_true) + 0.5 * tversky_loss(y_pred, y_true)

# Define the model architecture
model = smp.FPN(
    encoder_name=CFG.BACKBONE,
    encoder_weights='imagenet', # Start with imagenet, then load our weights
    in_channels=CFG.IN_CHANS,
    classes=1,
    activation=None,
)

# Load weights from the robust model onto the CPU first, then move to GPU
print(f"Loading pre-trained weights from {CFG.MODEL_LOAD_PATH}")
state_dict = torch.load(CFG.MODEL_LOAD_PATH, map_location=torch.device('cpu'))
model.load_state_dict(state_dict)
model.to(CFG.DEVICE)
print(f"Model with pre-trained weights moved to {CFG.DEVICE}")

# Optimizer with Differential Learning Rates
optimizer = torch.optim.AdamW([
    {'params': model.encoder.parameters(), 'lr': CFG.LR_ENCODER},
    {'params': model.decoder.parameters(), 'lr': CFG.LR_DECODER},
], weight_decay=CFG.WD)

# Scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG.EPOCHS, eta_min=1e-6)
scaler = GradScaler()

# --- Training and Validation Functions (reused from previous notebook) ---
def fbeta_score(y_true, y_pred, beta=0.5):
    tp = (y_true * y_pred).sum()
    fp = ((1 - y_true) * y_pred).sum()
    fn = (y_true * (1 - y_pred)).sum()
    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)
    fbeta = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall + 1e-6)
    return fbeta

def train_one_epoch(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc='Fine-Tuning')
    
    for i, (images, masks) in progress_bar:
        images = images.to(CFG.DEVICE)
        masks = masks.to(CFG.DEVICE)
        optimizer.zero_grad()
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, masks)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
        progress_bar.set_postfix(loss=total_loss / (i + 1))
    return total_loss / len(train_loader)

def validate(model, val_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, masks in tqdm(val_loader, desc='Validating'):
            images = images.to(CFG.DEVICE)
            with autocast():
                outputs = model(images)
            preds = torch.sigmoid(outputs).cpu()
            all_preds.append(preds)
            all_labels.append(masks)
    all_preds = torch.cat(all_preds).numpy().flatten()
    all_labels = torch.cat(all_labels).numpy().flatten()
    best_score = 0
    best_threshold = 0
    for threshold in np.arange(0.1, 0.9, 0.05):
        preds_binary = (all_preds > threshold).astype(int)
        score = fbeta_score(all_labels, preds_binary)
        if score > best_score:
            best_score = score
            best_threshold = threshold
    return best_score, best_threshold

Loading pre-trained weights from best_robust_model.pth
Model with pre-trained weights moved to cuda


/tmp/ipykernel_8093/3683003869.py:54: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [6]:
# --- 6. Main Fine-Tuning Loop ---

# First, get the dataloaders
train_loader, val_loader = get_dataloaders()

best_val_score = 0
epochs_no_improve = 0

for epoch in range(CFG.EPOCHS):
    print(f"\n--- Fine-Tuning Epoch {epoch+1}/{CFG.EPOCHS} ---")
    start_time = time.time()
    
    # Train for one epoch
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    
    # Apply learning rate scheduler step
    scheduler.step()
    
    # Validate the model
    val_score, best_thresh = validate(model, val_loader)
    
    elapsed_time = time.time() - start_time
    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val F0.5: {val_score:.4f} @ thresh {best_thresh:.2f} | Time: {elapsed_time:.2f}s")
    
    if val_score > best_val_score:
        print(f"Validation score improved from {best_val_score:.4f} to {val_score:.4f}. Saving model to {CFG.MODEL_SAVE_PATH}")
        best_val_score = val_score
        torch.save(model.state_dict(), CFG.MODEL_SAVE_PATH)
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"Validation score did not improve. Patience: {epochs_no_improve}/{CFG.PATIENCE}")
    
    if epochs_no_improve >= CFG.PATIENCE:
        print("Early stopping triggered.")
        break
        
    gc.collect()

print(f"\nFine-tuning complete. Best validation score: {best_val_score:.4f}")

--- Preparing Training Tiles ---
Processing fragment 1 for tile generation...


  Generated 1771 tiles (1093 with ink, 678 without ink). Randomizing tile order.

--- Preparing Validation Tiles ---
Processing fragment 2 for tile generation...


  Generated 5956 tiles (3797 with ink, 2159 without ink). Randomizing tile order.

--- Fine-Tuning Epoch 1/8 ---


Fine-Tuning:   0%|          | 0/443 [00:00<?, ?it/s]

  (Cache miss. Loading and normalizing fragment 1...)

...

Validating:  99%|█████████▉| 1474/1489 [01:14<00:00, 51.73it/s]

Validating:  99%|█████████▉| 1480/1489 [01:14<00:00, 51.24it/s]

Validating: 100%|█████████▉| 1486/1489 [01:15<00:00, 50.32it/s]

Validating: 100%|██████████| 1489/1489 [01:15<00:00, 19.81it/s]

Epoch 7 | Train Loss: 0.3975 | Val F0.5: 0.2750 @ thresh 0.10 | Time: 164.42s
Validation score did not improve. Patience: 3/3
Early stopping triggered.

Fine-tuning complete. Best validation score: 0.2759
